#### Task 12
Write a python class that estimate the camera pose given a planar marker, using homographies. 

In [1]:
import numpy as np
import cv2

class PoseEstimatorFromPlanarMarker:
    def __init__(self, K: np.ndarray):
        """
        K: (3x3) Camera intrinsic matrix.
        """
        self.K = K
        self.K_inv = np.linalg.inv(K)

    def estimate_pose(self, image_points: np.ndarray, world_points: np.ndarray):
        """
        Estimate the pose from 4+ 2D-3D correspondences assuming world_points lie on a plane.
        
        image_points: (N, 2) pixel coordinates
        world_points: (N, 3) 3D coordinates (Z=0 for all if planar)
        
        Returns:
            R (3x3): Rotation matrix
            t (3x1): Translation vector
        """
        # Make sure inputs are in correct format
        image_points = np.asarray(image_points, dtype=np.float32)
        world_points = np.asarray(world_points, dtype=np.float32)

        # Use only x and y for world points (assume Z=0)
        world_points_planar = world_points[:, :2]

        # Compute homography from world (planar) to image
        H, _ = cv2.findHomography(world_points_planar, image_points)

        # Decompose homography into R and t
        return self.decompose_homography(H)

    def decompose_homography(self, H: np.ndarray):
        """
        Decompose the homography into rotation and translation assuming a planar scene.
        """
        # Normalize homography
        H = H / np.linalg.norm(self.K_inv @ H[:, 0])
        H_cam = self.K_inv @ H

        h1 = H_cam[:, 0]
        h2 = H_cam[:, 1]
        h3 = np.cross(h1, h2)

        R = np.stack([h1, h2, h3], axis=1)
        U, _, Vt = np.linalg.svd(R)
        R = U @ Vt  # Ensure it's a valid rotation matrix (det(R) = 1)

        t = H_cam[:, 2].reshape(3, 1)

        return R, t


In [2]:
# Intrinsic matrix
K = np.array([[800, 0, 320],
              [0, 800, 240],
              [0,   0,   1]])

# 3D points on the marker (Z = 0 for all)
world_pts = np.array([
    [0, 0, 0],
    [1, 0, 0],
    [1, 1, 0],
    [0, 1, 0]
])

# Detected corners of the marker in the image (pixels)
image_pts = np.array([
    [320, 240],
    [400, 240],
    [400, 320],
    [320, 320]
])

pose_estimator = PoseEstimatorFromPlanarMarker(K)
R, t = pose_estimator.estimate_pose(image_pts, world_pts)

print("Rotation matrix:\n", R)
print("Translation vector:\n", t)


Rotation matrix:
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Translation vector:
 [[-1.28785871e-15]
 [ 1.99840144e-16]
 [ 1.00000000e+01]]
